In [1]:
import numpy as np
import cv2
# from scipy.misc import imresize
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from keras.models import model_from_json

import pickle
import matplotlib.image as pimg

## Load network model

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# Load Keras model
json_file = open('full_CNN_model3.json', 'r')
json_model = json_file.read()
json_file.close()
model = model_from_json(json_model)
model.load_weights('full_CNN_model3.h5')

In [4]:
# Class to average lanes with
class Lanes():
    def __init__(self):
        self.recent_fit = []
        self.avg_fit = []

## Define to predict the result with CNN network model

In [5]:
def road_lines_image(imageIn):
    """ Takes in a road image, re-sizes for the model,
    predicts the lane to be drawn from the model in G color,
    recreates an RGB image of a lane and merges with the
    original road image.
    """
    #crop to 720x1280, img[y: y + h, x: x + w], 300:940
    t_image = imageIn[230:950, 0:1280]

    size = (640, 1280, 3)
    
    image = cv2.resize(t_image, (640, 1280))
    
    # Get image ready for feeding into model
    # small_img = imresize(image, (80, 160, 3))
    small_img = cv2.resize(image, (160, 80))
    small_img = np.array(small_img)
    small_img = small_img[None,:,:,:]

    # Make prediction with neural network (un-normalize value by multiplying by 255)
    prediction = model.predict(small_img)[0] * 255

    # Add lane prediction to list for averaging
    lanes.recent_fit.append(prediction)
    # Only using last five for average
    if len(lanes.recent_fit) > 5:
        lanes.recent_fit = lanes.recent_fit[1:]

    # Calculate average detection
    lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)

    # Generate fake R & B color dimensions, stack with G
    blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)
    lane_drawn = np.dstack((blanks, lanes.avg_fit, blanks))

    # Re-size to match the original image
    # lane_image = imresize(lane_drawn, image.shape)
#     cv2_imshow(lane_drawn)
    
    # print(image.shape)
    lane_image = cv2.resize(lane_drawn, (640, 1280))
    # print(lane_image.shape)
    # cv2_imshow(lane_image)
    lane_img = np.asarray(lane_image)
    # Merge the lane drawing onto the original image
    # result = cv2.addWeighted(image, 1, lane_img, 1, 0.0)
    result = np.uint8(0.35*(image)+0.65*(lane_img))
    # cv2_imshow(result)
    return result

## Set detecting data

In [6]:
lanes = Lanes()

## Draw image and load the detection result

In [7]:
def road_lines_fc(imageIn):
    img_det = road_lines_image(imageIn)
    cv2.imshow("result", img_det)  

    cv2.waitKey(1)

In [8]:
videoFeed = cv2.VideoCapture("test_video7.mp4")

while videoFeed.isOpened() :
    (status, image) = videoFeed.read()
    img_det = road_lines_image(image)
    cv2.imshow("result",img_det)  

    k = cv2.waitKey(1)
    if k == 27:
        break
